##### Most difficult thing for me was to start at first, I already had a plan to use LSTM networks and wrote Dataset but everything came to Data at last.  I was given a huge dataset massive 5 GB around thats also in json file, and lost hope to start with notebook,
##### But evantually now I had a solution, to not only convert but also compress.  I went to kaggle created a notebook uploaded the json(took time) then ran import json import csv
##### and in the same file droped unnecessary columns, clicked save run and now I am using the csv file only of 2 GB
https://www.kaggle.com/bibhabasumohapatra/amazon-dataset-csv-generator/data?select=PolynomialInternshipDrive2022.csv
now, https://www.youtube.com/watch?v=oreIJQZ40H0&t=636s

In [1]:
import torch
import transformers
import torch.nn as nn

In [2]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 10
TOKENIZER = transformers.BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True, max_len = MAX_LEN , padding = 'max_length')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

# Dataset.py

In [3]:
class AmazonDataset(torch.utils.data.Dataset):
    def __init__(self, review, target):
        #

        self.review = review
        self.target = target
        self.tokenizer = TOKENIZER
        self.max_len = MAX_LEN
        
    def __len__(self):
        return len(self.review)
    
    def __getitem__(self, item):
        
        review = str(self.review[item])
        review = " ".join(review.split())
        
        inputs = self.tokenizer.encode_plus(
        review,
        None,
        add_special_tokens=True,
        max_length=self.max_len,
        pad_to_max_length=True,)
        
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        token_type_ids = inputs["token_type_ids"]
        
        return {
        "ids": torch.tensor(
        ids, dtype=torch.long
        ),
        "mask": torch.tensor(
        mask, dtype=torch.long
        ),
        "token_type_ids": torch.tensor(
        token_type_ids, dtype=torch.long
        ),
        "targets": torch.tensor(
        self.target[item], dtype=torch.float
        )
        }

In [4]:
## LOSS FUNCTION

def loss_fn(outputs, targets):
    return nn.BCEWithLogitsLoss()(outputs, targets.view(-1, 1))

## Engine.py

In [5]:
def train(data_loader, model, optimizer, device, scheduler):
    
    model.train()
    for d in data_loader:
 
 
        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets = d["targets"]
        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(
        ids=ids,
        mask=mask,
        token_type_ids=token_type_ids
        )

        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
    
    
def evaluate(data_loader, model, device):
 
    model.eval()
    
    fin_targets = []
    fin_outputs = []

    with torch.no_grad():
    
        for d in data_loader:
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            targets = d["targets"]
            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)
            outputs = model(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids
            )
            targets = targets.cpu().detach()
            fin_targets.extend(targets.numpy().tolist())
            
        outputs = torch.sigmoid(outputs).cpu().detach()
        fin_outputs.extend(outputs.numpy().tolist())
    return fin_outputs, fin_targets

## Model.py

In [6]:
class BERTModel(nn.Module):
    def __init__(self):
        super(BERTModel, self).__init__()
        self.bert = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        self.bert_drop = nn.Dropout(0.3)
        self.out = nn.Linear(768, 1)
        
    def forward(self, ids, mask, token_type_ids):
        
        _, x = self.bert(
        ids,
        attention_mask=mask,
        token_type_ids=token_type_ids
        )
        x = self.bert_drop(x)
        x = self.out(x)            
        return x

# Main.py

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

In [ ]:
df = pd.read_csv('../input/amazon-dataset-csv-generator/PolynomialInternshipDrive2022.csv')
#######################################################
df_train, df_valid = train_test_split(df, test_size=0.3, random_state=42, stratify=df.overall.values)

df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)

train_dataset = AmazonDataset(review=df_train.review.values, target=df_train.overall.values)
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=0,)# collate_fn=collate_fn)

valid_dataset = AmazonDataset(review=df_valid.review.values,target=df_valid.overall.values)
valid_data_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=VALID_BATCH_SIZE, num_workers=0,)# collate_fn=collate_fn)

device = torch.device("cuda")
model = BERTModel()
model.to('cuda')

param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
    {
 "params": [
 p for n, p in param_optimizer if
 not any(nd in n for nd in no_decay)
 ],
 "weight_decay": 0.001,
 },
 {
 "params": [
 p for n, p in param_optimizer if
 any(nd in n for nd in no_decay)
 ],
 "weight_decay": 0.0,
 },
 ]

num_train_steps = int(
 len(df_train) / TRAIN_BATCH_SIZE * EPOCHS
 )

optimizer = AdamW(optimizer_parameters, lr=3e-5)

scheduler = get_linear_schedule_with_warmup(
 optimizer,
 num_warmup_steps=0,
 num_training_steps=num_train_steps
 )

best_accuracy = 0
for epoch in range(EPOCHS):
    train(
    train_data_loader, model, optimizer, device, scheduler
    )
outputs, targets = eval_fn(
    valid_data_loader, model, device
 )

outputs = np.array(outputs) >= 0.5
accuracy = metrics.accuracy_score(targets, outputs)
print(f"Accuracy Score = {accuracy}")
if accuracy > best_accuracy:
    torch.save(model.state_dict(), MODEL_PATH)
    best_accuracy = accuracy

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max 

In [ ]:
df_train